In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install pandas nltk librosa numpy transformers scikit-learn torch tensorflow accelerate

In [3]:
# os.environ['KAGGLE_USERNAME'] = 'your_username'
# os.environ['KAGGLE_KEY'] = 'your_key'

In [5]:
# import kaggle
import pandas as pd

import shutil
import os

import nltk
import string

import librosa
import numpy as np

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
# Use a smaller model
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, RandomSampler, SequentialSampler

from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import logging
from torch.cuda.amp import GradScaler

from collections import Counter

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

import spacy
from sklearn.utils import resample


# import matplotlib.pyplot as plt


In [5]:
# # Change to the desired directory
# os.chdir('/content/drive/My Drive/Test Code')

# 3. Set Up Logging


In [6]:
# # Ensure old log file is removed before creating a new one
# file_path = 'training.log'
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"File '{file_path}' has been deleted.")
# else:
#     print(f"File '{file_path}' does not exist.")

# # Set up logging to record training progress
# logging.basicConfig(filename='training.log', level=logging.INFO)
# print("Logging file created")


File 'training.log' has been deleted.
Logging file created


# Contexual Emotion Detection

## 1. Download Dataset

In [7]:
# # Download dataset
# kaggle.api.dataset_download_files('pashupatigupta/emotion-detection-from-text', path='Data/.', unzip=True)


In [8]:
# # Load dataset
# text_dataset  = pd.read_csv('/kaggle/input/d/pashupatigupta/emotion-detection-from-text/tweet_emotions.csv')
# print(text_dataset.head())
# print(text_dataset['sentiment'].unique())

     tweet_id   sentiment                                            content
0  1956967341       empty  @tiffanylue i know  i was listenin to bad habi...
1  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...
2  1956967696     sadness                Funeral ceremony...gloomy friday...
3  1956967789  enthusiasm               wants to hang out with friends SOON!
4  1956968416     neutral  @dannycastillo We want to trade with someone w...
['empty' 'sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun'
 'hate' 'happiness' 'boredom' 'relief' 'anger']


In [9]:
# # Download dataset
# kaggle.api.dataset_download_files('ritika0111/emotion-detection-dataset', path='.', unzip=True)


In [10]:
# # List files in the extracted directory
# print(os.listdir('/kaggle/input/emotion-detection-dataset'))

['fear', 'disgusting', 'angry', 'neutral', 'sad', 'happy']


## 2. Preprocess Data

Text Data: Loaded from Data/tweet_emotions.csv, tokenized, and normalized.

Audio Data: Loaded from Data/emotion-detection-dataset, features extracted (MFCCs), and labeled.

Optional: Saved the preprocessed data for future use.

#### Text Data

In [11]:
# # Load the English model from SpaCy
# nlp = spacy.load('en_core_web_sm')

In [12]:
# # Additional text preprocessing
# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()

In [13]:
# # Label mapping
# label_mapping = {
#     'anger': 'angry',
#     'hate': 'disgusting',
#     'worry': 'fear',
#     'happiness': 'happy',
#     'fun': 'happy',
#     'enthusiasm': 'happy',
#     'neutral': 'neutral',
#     'sadness': 'sad',
#     'empty': 'sad'
# }

# # Map text dataset labels to audio dataset labels
# text_dataset['mapped_sentiment'] = text_dataset['sentiment'].map(label_mapping)

# # Filter out rows with labels that do not have a corresponding audio label
# filtered_text_dataset = text_dataset.dropna(subset=['mapped_sentiment'])


In [14]:
# print(filtered_text_dataset['mapped_sentiment'].unique())

['sad' 'happy' 'neutral' 'fear' 'disgusting' 'angry']


In [15]:
# def preprocess_text(text):
#     # Use SpaCy for text processing: tokenization, lemmatization, and stopword removal
#     doc = nlp(text)
#     tokens = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
#     return tokens

# # Apply preprocessing
# filtered_text_dataset['tokens'] = filtered_text_dataset['content'].apply(preprocess_text)
# filtered_text_dataset['content'] = filtered_text_dataset['tokens'].apply(lambda x: ' '.join(x))
# print(filtered_text_dataset.head())

     tweet_id   sentiment                                      content  \
0  1956967341       empty  know listenin bad habit early start freakin   
1  1956967666     sadness           layin n bed headache ughhhh waitin   
2  1956967696     sadness               funeral ceremony gloomy friday   
3  1956967789  enthusiasm                        want hang friend soon   
4  1956968416     neutral                    want trade houston ticket   

  mapped_sentiment                                             tokens  
0              sad  [know, listenin, bad, habit, early, start, fre...  
1              sad          [layin, n, bed, headache, ughhhh, waitin]  
2              sad                [funeral, ceremony, gloomy, friday]  
3            happy                         [want, hang, friend, soon]  
4          neutral                     [want, trade, houston, ticket]  


/tmp/ipykernel_36/1230115163.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_text_dataset['tokens'] = filtered_text_dataset['content'].apply(preprocess_text)
/tmp/ipykernel_36/1230115163.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_text_dataset['content'] = filtered_text_dataset['tokens'].apply(lambda x: ' '.join(x))


In [16]:
# # Balancing the dataset by oversampling minority classes
# def balance_dataset(df, target_column):
#     class_counts = df[target_column].value_counts()
#     max_class = class_counts.max()

#     dfs = [df]
#     for class_ in class_counts.index:
#         df_class = df[df[target_column] == class_]
#         if len(df_class) < max_class:
#             df_class = resample(df_class, replace=True, n_samples=max_class, random_state=42)
#         dfs.append(df_class)

#     return pd.concat(dfs)

In [17]:
# balanced_text_dataset = balance_dataset(filtered_text_dataset, 'mapped_sentiment')
# print(balanced_text_dataset['mapped_sentiment'].value_counts())

mapped_sentiment
neutral       17276
fear          17097
happy         16382
sad           14630
disgusting     9961
angry          8748
Name: count, dtype: int64


In [18]:
# # Ensure old log file is removed before creating a new one
# file_path = 'processed_text_data.csv'
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"File '{file_path}' has been deleted.")
# else:
#     print(f"File '{file_path}' does not exist.")

# # Save preprocessed text data
# balanced_text_dataset.to_csv('processed_text_data.csv', index=False)


File 'processed_text_data.csv' does not exist.


#### Audio Data

In [19]:
# # Function to extract MFCC features from audio files
# def extract_features(audio_path):
#     y, sr = librosa.load(audio_path)
#     mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
#     return np.mean(mfccs.T, axis=0)

In [20]:
# # Directory containing audio files
# audio_dir = '/kaggle/input/emotion-detection-dataset'
# emotions = list(label_mapping.values())

In [21]:
# audio_features = []
# audio_labels = []

# for emotion in emotions:
#     emotion_dir = os.path.join(audio_dir, emotion)
#     for file in os.listdir(emotion_dir):
#         if file.endswith('.wav'):
#             file_path = os.path.join(emotion_dir, file)
#             features = extract_features(file_path)
#             audio_features.append(features)
#             audio_labels.append(emotion)

In [22]:
# # Convert to numpy arrays
# audio_features = np.array(audio_features)
# audio_labels = np.array(audio_labels)
# print(audio_features.shape, audio_labels.shape)

(9783, 13) (9783,)


In [23]:
# # Ensure old log file is removed before creating a new one
# file_path = 'processed_audio_features.npy'
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"File '{file_path}' has been deleted.")
# else:
#     print(f"File '{file_path}' does not exist.")

# # Save preprocessed audio data
# np.save('processed_audio_features.npy', audio_features)
# np.save('audio_labels.npy', audio_labels)

File 'processed_audio_features.npy' does not exist.


## Train Emotion Detection Models

#### Text Emotion Detection
Training Text Emotion Detection Model:

You can use models like Distilbert for text emotion detection.

In [11]:
balanced_text_dataset = pd.read_csv('processed_text_data.csv')

In [12]:
print(balanced_text_dataset.head())

     tweet_id   sentiment                                      content  \
0  1956967341       empty  know listenin bad habit early start freakin   
1  1956967666     sadness           layin n bed headache ughhhh waitin   
2  1956967696     sadness               funeral ceremony gloomy friday   
3  1956967789  enthusiasm                        want hang friend soon   
4  1956968416     neutral                    want trade houston ticket   

  mapped_sentiment                                             tokens  
0              sad  ['know', 'listenin', 'bad', 'habit', 'early', ...  
1              sad  ['layin', 'n', 'bed', 'headache', 'ughhhh', 'w...  
2              sad        ['funeral', 'ceremony', 'gloomy', 'friday']  
3            happy                 ['want', 'hang', 'friend', 'soon']  
4          neutral             ['want', 'trade', 'houston', 'ticket']  


In [13]:
print(balanced_text_dataset['mapped_sentiment'].unique())

['sad' 'happy' 'neutral' 'fear' 'disgusting' 'angry']


In [14]:
# Split data
X = balanced_text_dataset['content']
y = balanced_text_dataset['mapped_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


In [24]:
# # Use pre-downloaded model in Kaggle
# tokenizer = DistilBertTokenizer.from_pretrained('/kaggle/input/huggingface-distilbert/distilbert-base-uncased')


OSError: Incorrect path_or_model_id: '/kaggle/input/huggingface-distilbert/distilbert-base-uncased'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [22]:
# def encode_data(texts, tokenizer, max_len):
#     input_ids = []
#     attention_masks = []

#     for text in texts:
#         if isinstance(text, str):  # Ensure the text is a string
#             encoded_dict = tokenizer.encode_plus(
#                 text,
#                 add_special_tokens=True,
#                 max_length=max_len,
#                 padding='max_length',
#                 return_attention_mask=True,
#                 return_tensors='pt',
#                 truncation=True
#             )
#             input_ids.append(encoded_dict['input_ids'].squeeze(0))  # Remove batch dimension
#             attention_masks.append(encoded_dict['attention_mask'].squeeze(0))  # Remove batch dimension
#         else:
#             # Handle non-string text by appending empty tensors
#             empty_tensor = torch.zeros((max_len,), dtype=torch.long)
#             input_ids.append(empty_tensor)
#             attention_masks.append(empty_tensor)

#     input_ids = torch.stack(input_ids)
#     attention_masks = torch.stack(attention_masks)

#     return input_ids, attention_masks


In [15]:
# # Label encoding
# label_encoder = LabelEncoder()
# y_train_labels = label_encoder.fit_transform(y_train)
# y_test_labels = label_encoder.transform(y_test)

# # Convert labels to tensors
# train_labels = torch.tensor(y_train_labels, dtype=torch.long)
# test_labels = torch.tensor(y_test_labels, dtype=torch.long)

In [23]:
# # Encode the training and testing data
# train_inputs, train_masks = encode_data(X_train, tokenizer, max_len=225)
# test_inputs, test_masks = encode_data(X_test, tokenizer, max_len=225)

KeyboardInterrupt: 

In [ ]:
# # Create DataLoader for training and testing sets
# batch_size = 16
# train_data = TensorDataset(train_inputs, train_masks, train_labels)
# train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=batch_size)

# test_data = TensorDataset(test_inputs, test_masks, test_labels)
# test_dataloader = DataLoader(test_data, sampler=SequentialSampler(test_data), batch_size=batch_size)

In [33]:
# print("Unique labels in training set:", np.unique(y_train_labels))
# print("Unique labels in testing set:", np.unique(y_test_labels))


Unique labels in training set: [0 1 2 3 4 5]
Unique labels in testing set: [0 1 2 3 4 5]


In [16]:
# # Adjust model architecture with dropout
# class DistilBertForSequenceClassificationWithDropout(DistilBertForSequenceClassification):
#     def __init__(self, config):
#         super().__init__(config)
#         self.dropout = torch.nn.Dropout(0.3)  # Add dropout with 30% probability

#     def forward(self, input_ids, attention_mask=None, labels=None):
#         outputs = self.distilbert(input_ids, attention_mask=attention_mask)
#         pooled_output = outputs[0][:, 0]
#         pooled_output = self.dropout(pooled_output)  # Apply dropout
#         logits = self.classifier(pooled_output)

#         loss = None
#         if labels is not None:
#             loss_fct = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights).to(device))
#             loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

#         return torch.nn.functional.softmax(logits, dim=-1) if loss is None else loss, logits


In [35]:
# # Initialize the new model with dropout
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = DistilBertForSequenceClassificationWithDropout.from_pretrained("/kaggle/input/huggingface-distilbert/distilbert-base-uncased", num_labels=len(set(y_train))).to(device)


Some weights of DistilBertForSequenceClassificationWithDropout were not initialized from the model checkpoint at /kaggle/input/huggingface-distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# # Calculate class weights
# class_counts = Counter(y_train_labels)
# total_samples = len(y_train_labels)
# class_weights = [total_samples / class_counts[i] for i in range(len(set(y_train_labels)))]
# epochs = 25

# # Optimizer and scheduler
# optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
# total_steps = len(train_dataloader) * epochs / 2
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [37]:
# def train_model_mixed_precision(model, dataloader, optimizer, scheduler, epochs, device, scaler):
#     model.train()

#     for epoch in range(epochs):
#         print(f'Epoch {epoch + 1}/{epochs}')
#         logging.info(f'Epoch {epoch + 1}/{epochs}')

#         total_loss = 0
#         step = 1
#         for batch in dataloader:
#             input_ids, attention_masks, labels = batch
#             input_ids = input_ids.to(device)
#             attention_masks = attention_masks.to(device)
#             labels = labels.to(device)

#             optimizer.zero_grad()

#             with torch.cuda.amp.autocast():
#                 outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
#                 loss = outputs[0]  # Access the loss from the tuple
#                 total_loss += loss.item()

#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()
#             scheduler.step()

#             # Print and log the loss for each batch
#             if step % 30 == 0:  # Adjust the frequency of logging if necessary
#                 print(f"Epoch {epoch + 1}/{epochs} | Batch {step + 1}/{len(dataloader)} | Loss: {loss.item():.4f}")
#                 logging.info(f"Epoch {epoch + 1}/{epochs} | Batch {step + 1}/{len(dataloader)} | Loss: {loss.item():.4f}")

#             step += 1

#         avg_train_loss = total_loss / len(dataloader)
#         print(f'Average training loss: {avg_train_loss:.2f}')
#         logging.info(f'Average training loss: {avg_train_loss:.2f}')

#         if avg_train_loss < 0.001:
#             break

#     print('Training complete')
#     logging.info('Training complete')


In [38]:
# # Initialize scaler for mixed precision training
# scaler = GradScaler()


# # Train the model
# train_model_mixed_precision(model, train_dataloader, optimizer, scheduler, epochs, device=device, scaler=scaler)

/tmp/ipykernel_36/3149539752.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_36/3234688868.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/25
Epoch 1/25 | Batch 31/3680 | Loss: 1.8215
Epoch 1/25 | Batch 61/3680 | Loss: 1.7143
Epoch 1/25 | Batch 91/3680 | Loss: 1.8127
Epoch 1/25 | Batch 121/3680 | Loss: 1.6770
Epoch 1/25 | Batch 151/3680 | Loss: 1.7113
Epoch 1/25 | Batch 181/3680 | Loss: 1.6437
Epoch 1/25 | Batch 211/3680 | Loss: 1.6113
Epoch 1/25 | Batch 241/3680 | Loss: 1.7286
Epoch 1/25 | Batch 271/3680 | Loss: 1.5656
Epoch 1/25 | Batch 301/3680 | Loss: 1.6014
Epoch 1/25 | Batch 331/3680 | Loss: 1.4939
Epoch 1/25 | Batch 361/3680 | Loss: 1.3276
Epoch 1/25 | Batch 391/3680 | Loss: 1.4052
Epoch 1/25 | Batch 421/3680 | Loss: 1.6882
Epoch 1/25 | Batch 451/3680 | Loss: 1.3678
Epoch 1/25 | Batch 481/3680 | Loss: 1.3150
Epoch 1/25 | Batch 511/3680 | Loss: 1.5649
Epoch 1/25 | Batch 541/3680 | Loss: 0.7916
Epoch 1/25 | Batch 571/3680 | Loss: 1.5370
Epoch 1/25 | Batch 601/3680 | Loss: 1.3169
Epoch 1/25 | Batch 631/3680 | Loss: 1.2132
Epoch 1/25 | Batch 661/3680 | Loss: 0.7390
Epoch 1/25 | Batch 691/3680 | Loss: 0.7799
Epo

In [39]:
# # Save the model's state dictionary using torch.save
# output_dir = '/kaggle/working/text_model/'

# # Remove the previous directory if it exists
# if os.path.exists(output_dir):
#     shutil.rmtree(output_dir)
#     print("Previous directory deleted")

# # Create a new directory
# os.makedirs(output_dir)
# print("New directory created")

# # Save the model's state dictionary
# model_path = os.path.join(output_dir, 'model_state_dict.pt')
# torch.save(model.state_dict(), model_path)

# # Save the tokenizer
# tokenizer.save_pretrained(output_dir)

# print(f"Model state dictionary saved to {model_path}")
# logging.info(f"Model state dictionary saved to {model_path}")


New directory created
Model state dictionary saved to ./model_save/text_model/model_state_dict.pt


In [18]:
# # Define the directory where the model is saved
# output_dir = './model_save/text_model/'

# # Load the tokenizer
# tokenizer = DistilBertTokenizer.from_pretrained(output_dir)

# # Initialize the model with the appropriate configuration
# model = DistilBertForSequenceClassificationWithDropout.from_pretrained("distilbert-base-uncased", num_labels=len(set(y_train)))

# # Load the model's state dictionary
# model_path = os.path.join(output_dir, 'model_state_dict.pt')
# model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# # Move the model to the same device you used for training (e.g., GPU if available)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# print("Model and tokenizer loaded successfully.")


Some weights of DistilBertForSequenceClassificationWithDropout were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and tokenizer loaded successfully.


In [19]:

# def evaluate(model, dataloader, device):
#     model.eval()
#     total_eval_loss = 0
#     total_eval_accuracy = 0

#     for batch in dataloader:
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)

#         with torch.no_grad():
#             outputs = model(b_input_ids,
#                             attention_mask=b_input_mask,
#                             labels=b_labels)
            
#             # Unpack the outputs tuple
#             loss, logits = outputs[:2]

#         total_eval_loss += loss.item()

#         # Calculate accuracy
#         preds = torch.argmax(logits, dim=1).flatten()
#         accuracy = (preds == b_labels).cpu().numpy().mean()
#         total_eval_accuracy += accuracy

#     avg_eval_loss = total_eval_loss / len(dataloader)
#     avg_eval_accuracy = total_eval_accuracy / len(dataloader)
    
#     print(f"Evaluation Loss: {avg_eval_loss}")
#     print(f"Evaluation Accuracy: {avg_eval_accuracy}")

#     return avg_eval_loss, avg_eval_accuracy


In [20]:
# # Evaluate the model on the test set
# evaluate(model, test_dataloader, device)

NameError: name 'test_dataloader' is not defined

In [43]:
# def predict(text, model, tokenizer, label_encoder, device, max_len=128):
#     model.eval()
    
#     # Tokenize and encode the text
#     inputs = tokenizer.encode_plus(
#         text,
#         add_special_tokens=True,
#         max_length=max_len,
#         pad_to_max_length=True,
#         return_tensors='pt',
#         truncation=True
#     )
    
#     input_ids = inputs['input_ids']
#     masks = inputs['attention_mask']

#     with torch.no_grad():
#         outputs = model(input_ids.to(device), attention_mask=masks.to(device))
        
#         # Unpack the logits from the model outputs
#         logits = outputs[0]
#         pred = torch.argmax(logits, dim=1).flatten()
    
#     label = label_encoder.inverse_transform(pred.cpu().numpy())
#     return label[0]



In [56]:
# # Example prediction
# example_text = "I am angry mom."
# predicted_emotion = predict(example_text, model, tokenizer, label_encoder, device)
# print(f"Predicted Emotion: {predicted_emotion}")

Predicted Emotion: sad


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


#### Select or Train an Emotion-Aware TTS Model

You will need a TTS model that can generate speech with different emotional tones.

#### Map Detected Emotion to Speech Emotion

Map the detected emotion from the text to the corresponding emotion category in your TTS model or dataset.

#### Generate Speech Based on Detected Emotion

Once the emotion is detected and mapped, use the TTS model to generate speech that matches the detected emotion.